In [1]:
import os,time
os.environ['CUDA_VISIBLE_DEVICES']='0'
import copy
import logging
import numpy as np
import pandas as pd
from datetime import datetime,timedelta,date
import time
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModel, get_linear_schedule_with_warmup, \
    default_data_collator, TrainingArguments, Trainer
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances
from scipy.stats import pearsonr, spearmanr
from transformers import AutoTokenizer
from transformers import BertConfig, BertTokenizer,BertTokenizerFast
from datasets import Dataset
from typing import List,Union
import tqdm
import re
import json

os.chdir('/home/stops/Work_space/NLP_work/Med_assit_chatglm')

from db_config_taiyi import DB ## load data from pgsql

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(levelname)s - %(message)s')
logger.info('Starting')



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/stops/miniconda3/envs/LLM_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/stops/miniconda3/envs/LLM_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


2024-01-03 15:00:07,098 - INFO : Starting


In [2]:
##################################
## Instruct data
##################################

data_file='/home/stops/Work_space/NLP_models/train_2M_CN/train_2M_CN.json'

belle_instruct_data=[]
with open(data_file,'r') as f :
    for sub_data in f.readlines()[:50]:
        sub_data_dict=json.loads(sub_data)
        belle_instruct_data.append(sub_data_dict)
logger.info(f'belle_instruct_data nums: {len(belle_instruct_data)}')
logger.info(f'examples: {belle_instruct_data[:2]}')

2024-01-02 17:23:08,350 - INFO : belle_instruct_data nums: 50
2024-01-02 17:23:08,351 - INFO : examples: [{'instruction': '对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  print(add(2,3))', 'input': '', 'output': '这段代码的问题是在函数内部， `print` 语句出现在 `return` 语句之后，也就是说函数在 `return` 语句处结束，因此 `print` 语句永远不会被执行。\n要解决这个问题，只需在 `return` 语句之前移到 `print` 语句。修改后的代码如下所示：\n```\ndef add(a, b):\n    print(a + b)\n    return a + b\nadd(2, 3)\n```\n这样就可以在调用 `add` 函数时打印出结果，并在函数内部返回 `a + b` 值，使其在需要时可以接收和使用。同时，注意在调用 `add` 函数时，不需要再使用 `print` 方法了。'}, {'instruction': '为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。\n', 'input': '', 'output': '为一个新的智能手机应用程序起一个优秀的名称可以考虑以下几点：\n1. 名称应该简单易记，不宜太长；\n2. 名称应该与应用程序的主要功能或特点相关；\n3. 名称应该引起人们的兴趣和好奇心。\n举个例子，如果我们的应用程序是一款健康管理应用程序，那么可以考虑以下几个名称：\n1. HealthBoss\n2. FitMeUp\n3. HealthMate\n4. BodyBest\n5. HealthLab\n其中，HealthBoss强调应用程序可帮助用户成为健康的老板，FitMeUp强调应用程序可帮助用户达到健康生活的目标，HealthMate强调应用程序是用户健康的伙伴，BodyBest强调应用程序可通过健康管理帮助用户获得最佳身体状态，HealthLab强调应用程序可帮助用户进行健康诊断和实验。\n一句简短的介绍可包含以下几个要素：\n1. 应用程序的名称；\n2. 应用程序的主要功

In [3]:
belle_instruct_df=pd.DataFrame.from_dict(belle_instruct_data)
belle_instruct_df=belle_instruct_df.applymap(lambda x:x.strip())
print(belle_instruct_df.shape)
print(belle_instruct_df.head())
print(belle_instruct_df.isnull().sum())

(50, 3)
                                         instruction input  \
0  对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  p...         
1                     为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。         
2  在给定的代码示例中，修复一个特定的错误。\n以下是一个Python代码示例：\ndef su...         
3  根据文本生成一段简短的摘要。\n文本：据报道，乔治·华盛顿于1732年2月22日出生在维珍尼...         
4                                  回答下面这个问题：什么是人工智能？         

                                              output  
0  这段代码的问题是在函数内部， `print` 语句出现在 `return` 语句之后，也就是...  
1  为一个新的智能手机应用程序起一个优秀的名称可以考虑以下几点：\n1. 名称应该简单易记，不宜...  
2  该Python代码中的错误是调用sum()函数时传递了3个参数，而该函数只接受2个参数。因此...  
3                     乔治·华盛顿出生在1732年2月22日，位于维吉尼亚殖民地。  
4  人工智能（Artificial Intelligence，简称 AI）是一种旨在模拟、复制、...  
instruction    0
input          0
output         0
dtype: int64


In [4]:
belle_instruct_df.loc[:,'instruct_len']=belle_instruct_df.apply(lambda x:len(x['instruction']),axis=1)
print(belle_instruct_df['instruct_len'].describe(percentiles=[0.01,0.1,0.2,0.25,0.5,0.75,0.8,0.9,0.9999]))

count      50.000000
mean       47.440000
std        24.964595
min        11.000000
1%         13.450000
10%        22.800000
20%        28.400000
25%        30.250000
50%        43.000000
75%        52.000000
80%        61.000000
90%        80.900000
99.99%    131.853000
max       132.000000
Name: instruct_len, dtype: float64


In [5]:
select_instruct_df=belle_instruct_df.loc[belle_instruct_df['instruct_len']<600,['instruction','output']].copy()
print(select_instruct_df.shape)
print(select_instruct_df.head())

(50, 2)
                                         instruction  \
0  对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  p...   
1                     为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。   
2  在给定的代码示例中，修复一个特定的错误。\n以下是一个Python代码示例：\ndef su...   
3  根据文本生成一段简短的摘要。\n文本：据报道，乔治·华盛顿于1732年2月22日出生在维珍尼...   
4                                  回答下面这个问题：什么是人工智能？   

                                              output  
0  这段代码的问题是在函数内部， `print` 语句出现在 `return` 语句之后，也就是...  
1  为一个新的智能手机应用程序起一个优秀的名称可以考虑以下几点：\n1. 名称应该简单易记，不宜...  
2  该Python代码中的错误是调用sum()函数时传递了3个参数，而该函数只接受2个参数。因此...  
3                     乔治·华盛顿出生在1732年2月22日，位于维吉尼亚殖民地。  
4  人工智能（Artificial Intelligence，简称 AI）是一种旨在模拟、复制、...  


In [6]:
instruct_prompt_text="""对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。
参考类别：[计算，逻辑与推理，代码，知识与百科，语言理解与抽取，上下文对话，生成与创作，角色扮演，安全指令攻击，任务规划，其他]，同时不要出现没有在上面的类别。


指令文本内容：
"{text}"

任务要求如下：
1.首先给出所属类别的分析过程。
2.再返回结果格式为json，{{"类别":[xxx]}}。
"""


In [7]:
check_user_first_text_result_list=select_instruct_df["instruction"].tolist()

user_first_text_department_result_list=[]
for sub_text in check_user_first_text_result_list:
    sub_u_f_text=instruct_prompt_text.format(text=sub_text)
    user_first_text_department_result_list.append(sub_u_f_text)

logger.info(f"user_first_text_department_result_list nums: {len(user_first_text_department_result_list)}")

2024-01-02 17:23:08,377 - INFO : user_first_text_department_result_list nums: 50


In [8]:
print(user_first_text_department_result_list[:2])



['对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。\n参考类别：[计算，逻辑与推理，代码，知识与百科，语言理解与抽取，上下文对话，生成与创作，角色扮演，安全指令攻击，任务规划，其他]，同时不要出现没有在上面的类别。\n\n指令文本内容：\n"对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  print(add(2,3))"\n\n任务要求如下：\n1.首先给出所属类别的分析过程。\n2.再返回结果格式为json，{"类别":[xxx]}。\n', '对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。\n参考类别：[计算，逻辑与推理，代码，知识与百科，语言理解与抽取，上下文对话，生成与创作，角色扮演，安全指令攻击，任务规划，其他]，同时不要出现没有在上面的类别。\n\n指令文本内容：\n"为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。"\n\n任务要求如下：\n1.首先给出所属类别的分析过程。\n2.再返回结果格式为json，{"类别":[xxx]}。\n']


In [9]:
###################
## load model
###################
model_path="/home/stops/Work_space/NLP_models/Qwen-14B-Chat/"


from transformers import AutoModel, AutoTokenizer,AutoModelForCausalLM
from transformers.generation.utils import GenerationConfig
from torch.nn.utils.rnn import pad_sequence
from torch.types import Number

def right_padding(sequences: [torch.Tensor], padding_value) -> torch.Tensor:
    return pad_sequence(sequences, batch_first=True, padding_value=padding_value)


def left_padding(sequences: [torch.Tensor], padding_value) -> torch.Tensor:
    return right_padding(
        [seq.flip(0) for seq in sequences],
        padding_value=padding_value,).flip(1)


def init_model():
    model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True,fp16=True).cuda()
    model.generation_config = GenerationConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path,use_fast=False,trust_remote_code=True)
    logger.info(f"load model path: {model_path}")
    return model, tokenizer


In [10]:
model, tokenizer = init_model()


2024-01-02 17:23:08,439 - WARNING : Try importing flash-attention for faster inference...
2024-01-02 17:23:08,439 - WARNING : Warning: import flash_attn rotary fail, please install FlashAttention rotary to get higher efficiency https://github.com/Dao-AILab/flash-attention/tree/main/csrc/rotary
2024-01-02 17:23:08,440 - WARNING : Warning: import flash_attn rms_norm fail, please install FlashAttention layer_norm to get higher efficiency https://github.com/Dao-AILab/flash-attention/tree/main/csrc/layer_norm
2024-01-02 17:23:08,440 - WARNING : Warning: import flash_attn fail, please install FlashAttention to get higher efficiency https://github.com/Dao-AILab/flash-attention


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

2024-01-02 17:24:30,325 - INFO : load model path: /home/stops/Work_space/NLP_models/Qwen-14B-Chat/


In [11]:
## Qwen_Chat_batch_infer

def Qwen_FAQ_batch_infer(batch_text_list:List[str]):
    if isinstance(batch_text_list,str):
        batch_text_list=[batch_text_list]
    im_start, im_end = "<|im_start|>", "<|im_end|>"
    im_start_tokens = [tokenizer.im_start_id]
    im_end_tokens = [tokenizer.im_end_id]
    nl_tokens = tokenizer.encode("\n")
    batch_input_ids=[]
    def _tokenize_str(role, content):
        return tokenizer.encode(
                role, allowed_special=set()
            ) + nl_tokens + tokenizer.encode(content, allowed_special=set())
    system="You are a helpful assistant."
    system_tokens_part = _tokenize_str("system", system)
    system_tokens = im_start_tokens + system_tokens_part + im_end_tokens
    for sub_text in batch_text_list:
        sub_input_ids=system_tokens+nl_tokens+ im_start_tokens+ _tokenize_str("user", sub_text)+ im_end_tokens \
            + nl_tokens+ im_start_tokens+ tokenizer.encode("assistant")+ nl_tokens
        batch_input_ids.append(torch.tensor(sub_input_ids, dtype=torch.long))
    batch_input_len=[len(item) for item in batch_input_ids]
    batch_max_len=max(batch_input_len)
    ## padding-strategy: LEFT, QWEN eos_token_id=pad_token_id=151643
    batch_input_tensor_ids=left_padding(batch_input_ids, padding_value=151643)
    test_batch_input_ids=batch_input_tensor_ids.to("cuda")
    batch_outputs = model.generate(test_batch_input_ids)
    batch_response_text_result_list=[]
    for sub_output  in batch_outputs:
        sub_response = tokenizer.decode(sub_output[batch_max_len:], skip_special_tokens=True)
        batch_response_text_result_list.append(sub_response)
    return batch_response_text_result_list


In [12]:
test_text_list=user_first_text_department_result_list[:2]
print("test_text_list: ",test_text_list)

s_time=time.time()
test_infer_texts=Qwen_FAQ_batch_infer(test_text_list)
print("batch cost time : {:.2f}".format(time.time()-s_time))
print("test_infer_texts: ",test_infer_texts)


test_text_list:  ['对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。\n参考类别：[计算，逻辑与推理，代码，知识与百科，语言理解与抽取，上下文对话，生成与创作，角色扮演，安全指令攻击，任务规划，其他]，同时不要出现没有在上面的类别。\n\n指令文本内容：\n"对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  print(add(2,3))"\n\n任务要求如下：\n1.首先给出所属类别的分析过程。\n2.再返回结果格式为json，{"类别":[xxx]}。\n', '对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。\n参考类别：[计算，逻辑与推理，代码，知识与百科，语言理解与抽取，上下文对话，生成与创作，角色扮演，安全指令攻击，任务规划，其他]，同时不要出现没有在上面的类别。\n\n指令文本内容：\n"为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。"\n\n任务要求如下：\n1.首先给出所属类别的分析过程。\n2.再返回结果格式为json，{"类别":[xxx]}。\n']
batch cost time : 3.57
test_infer_texts:  ['该指令文本主要是关于代码的问题。通过阅读文本可以发现其描述了一个函数定义和调用的过程，并且存在语法错误（缺少冒号）。因此，这个文本最适合的类别应该是“代码”。\n\n返回结果：\n{\n    "类别": ["代码"]\n}\n', '分析过程: 这个指令需要进行创新和生成任务，即给定一个目标（新的智能手机应用程序），要求产生相应的输出（名称和简短介绍）。因此，这个指令应该被归类为“生成与创作”类别。\n\n返回结果:\n{"类别": ["生成与创作"]}\n']


In [22]:
extract_dict_text="""[{]\n?.*?\n?[}]"""
test_result=re.findall(extract_dict_text,test_infer_texts[0])
print("test_result: ",test_result)


test_result:  ['{\n    "类别": ["代码"]\n}']


In [19]:
test_text="""这个指令的目的是生成和创作一个新的智能手机应用程序的名称和简介。因此，它应该属于“生成与创作”类别。

返回结果格式为：
```json
{
  "类别": ["生成与创作"]
}
```"""
extract_dict_text="""[{]\n?.*?\n?[}]"""
test_result=re.findall(extract_dict_text,test_text,re.DOTALL)
print("test_result: ",test_result)
print("test_result_dict: ",eval(test_result[0]))

test_result:  ['{\n  "类别": ["生成与创作"]\n}']
test_result_dict:  {'类别': ['生成与创作']}


In [23]:
####################
## 预测结果
####################
department_infer_detail_result=[]
department_infer_result=[]
batch_size=2
all_nums=len(user_first_text_department_result_list)
logger.info(f"run nums: {all_nums} , batch_size: {batch_size}")

for idx in range(0,all_nums,batch_size):
    if idx%10==0:
        logger.info("run step: {}, finished: {:.2%}".format(idx,idx/all_nums))
    sub_batch_texts=user_first_text_department_result_list[idx:(idx+batch_size)]
    sub_infer_texts=Qwen_FAQ_batch_infer(sub_batch_texts)
    extract_dict_text="""[{]\n?.*?\n?[}]"""
    for sub_text in sub_infer_texts:
        sub_result=re.findall(extract_dict_text,sub_text)
        department_infer_detail_result.append(sub_text)
        if sub_result:
            sub_result_text=sub_result[0]
        else:
            sub_result_text=None
        department_infer_result.append(sub_result_text)

logger.info(f"department_infer_detail_result nums: {len(department_infer_detail_result)}")
logger.info(f"examples: {department_infer_detail_result[:2]}")


logger.info(f"department_infer_result nums: {len(department_infer_result)}")
logger.info(f"examples: {department_infer_result[:2]}")


2024-01-02 17:36:20,068 - INFO : run nums: 50 , batch_size: 2
2024-01-02 17:36:20,069 - INFO : run step: 0, finished: 0.00%
2024-01-02 17:36:33,693 - INFO : run step: 10, finished: 20.00%
2024-01-02 17:36:54,216 - INFO : run step: 20, finished: 40.00%
2024-01-02 17:37:09,095 - INFO : run step: 30, finished: 60.00%
2024-01-02 17:37:22,461 - INFO : run step: 40, finished: 80.00%
2024-01-02 17:37:36,399 - INFO : department_infer_detail_result nums: 50
2024-01-02 17:37:36,399 - INFO : examples: ['1. 分析过程：\n   这段文本描述了一个函数定义和一个函数调用。它需要我们理解和执行这段代码，并找出可能存在的错误或问题。这涉及到代码的理解、执行和调试，因此该指令可以归入“代码”类别。\n\n2. 返回结果：\n   {"类别": "代码"}\n', '1. 分析过程：该指令的任务是为新的智能手机应用程序命名和撰写简短介绍，这涉及到创意和写作能力，所以主要涉及的类别可能是“生成与创作”。\n\n2. 返回结果：\n   {"类别": "生成与创作"}\n']
2024-01-02 17:37:36,400 - INFO : department_infer_result nums: 50
2024-01-02 17:37:36,400 - INFO : examples: ['{"类别": "代码"}', '{"类别": "生成与创作"}']


In [24]:
final_department_result_list=[]
for sub_text in department_infer_result:
    #print("sub_text: ",sub_text)
    try:
        sub_dict=eval(sub_text)
        #print("sub_dict: ",sub_dict)
        sub_raw_res=sub_dict["类别"]
        if isinstance(sub_raw_res,List):
            sub_res=sub_raw_res[0]
        else:
            sub_res=sub_raw_res
    except:
        sub_res=None
    final_department_result_list.append(sub_res)

logger.info(f"final_department_result_list nums: {len(final_department_result_list)}")
logger.info(f"examples: {final_department_result_list[:2]}")


2024-01-02 17:37:36,405 - INFO : final_department_result_list nums: 50
2024-01-02 17:37:36,405 - INFO : examples: ['代码', '生成与创作']


In [25]:


save_df=select_instruct_df.head(len(final_department_result_list)).copy()

save_df.loc[:,"detail_result"]=department_infer_detail_result
save_df.loc[:,"dict_result"]=department_infer_result
save_df.loc[:,"result"]=final_department_result_list

print(save_df.shape)
print(save_df.head())
print(save_df.isnull().sum())

save_mode=True
save_file="output_data/Instruct_clf_result_by_qwen_231221_df.xlsx"
if save_mode:
    save_df.to_excel(save_file,index=False)
    logger.info(f"data nums: {save_df.shape}, save file: {save_file}")




2024-01-02 17:37:36,424 - INFO : data nums: (50, 5), save file: output_data/Instruct_clf_result_by_qwen_231221_df.xlsx


(50, 5)
                                         instruction  \
0  对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  p...   
1                     为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。   
2  在给定的代码示例中，修复一个特定的错误。\n以下是一个Python代码示例：\ndef su...   
3  根据文本生成一段简短的摘要。\n文本：据报道，乔治·华盛顿于1732年2月22日出生在维珍尼...   
4                                  回答下面这个问题：什么是人工智能？   

                                              output  \
0  这段代码的问题是在函数内部， `print` 语句出现在 `return` 语句之后，也就是...   
1  为一个新的智能手机应用程序起一个优秀的名称可以考虑以下几点：\n1. 名称应该简单易记，不宜...   
2  该Python代码中的错误是调用sum()函数时传递了3个参数，而该函数只接受2个参数。因此...   
3                     乔治·华盛顿出生在1732年2月22日，位于维吉尼亚殖民地。   
4  人工智能（Artificial Intelligence，简称 AI）是一种旨在模拟、复制、...   

                                       detail_result      dict_result result  
0  1. 分析过程：\n   这段文本描述了一个函数定义和一个函数调用。它需要我们理解和执行这段...     {"类别": "代码"}     代码  
1  1. 分析过程：该指令的任务是为新的智能手机应用程序命名和撰写简短介绍，这涉及到创意和写作能...  {"类别": "生成与创作"}  生成与创作  
2  分析过程：该指令需要用户理解和执行Python代码，并识别和修复其中的错误。这涉及到对编程语...     {"类别": "代码"}     代码  
3 

In [2]:

def show_df(df):
    print(df.shape)
    print(df.head(2))
    print(df.isnull().sum())




In [3]:

################################
## Stage-2: 针对预测的结果进行格式处理
################################

label_file="output_data/Alpaca_gpt4_zh_instruct_label_data_5w_240102.xlsx"
label_df=pd.read_excel(label_file)

show_df(label_df)


(48301, 6)
  instruction                                             output  \
0  保持健康的三个提示。  以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游...   
1     三原色是什么？  三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在...   

                                               input  \
0  对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。\n参考类别：[计算，逻辑与...   
1  对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。\n参考类别：[计算，逻辑与...   

                                       detail_result               dict_label  \
0  这个指令文本的内容是关于提供保持健康的建议或提示，因此它可以被分类为“生成与创作”类别。这类...          {"类别": "生成与创作"}   
1  这个指令的意图是询问某个特定的知识点，即“三原色”。因此，它应该被归入“知识与百科”类别。在...  {\n    "类别": "知识与百科"\n}   

  label  
0     生  
1     知  
instruction         0
output              0
input               0
detail_result       0
dict_label       6295
label            6961
dtype: int64


In [7]:
single_label_result_list=[]
combine_label_result_list=[]

dict_label_data=label_df["dict_label"].tolist()

for sub_dict_text in dict_label_data:
    if pd.notnull(sub_dict_text):
        try:
            sub_dict_data=eval(sub_dict_text)
            sub_raw_res = sub_dict_data["类别"]
            if isinstance(sub_raw_res, List):
                sub_comb_label_string="+".join(sub_raw_res)
                sub_single_label_string = sub_raw_res[0]
            else:
                sub_comb_label_string=sub_raw_res
                sub_single_label_string = sub_raw_res
        except:
            sub_comb_label_string=None
            sub_single_label_string =None
    else:
        sub_comb_label_string=None
        sub_single_label_string =None
    single_label_result_list.append(sub_single_label_string)
    combine_label_result_list.append(sub_comb_label_string)
    
    
logger.info(f"single_label_result_list nums: {len(single_label_result_list)}")
logger.info(f"combine_label_result_list nums: {len(combine_label_result_list)}")
logger.info(f"Examples: {single_label_result_list[:5]}")
logger.info(f"Examples: {combine_label_result_list[:5]}")



2024-01-03 15:23:01,831 - INFO : single_label_result_list nums: 48301
2024-01-03 15:23:01,832 - INFO : combine_label_result_list nums: 48301
2024-01-03 15:23:01,832 - INFO : Examples: ['生成与创作', '知识与百科', '知识与百科', None, '故事/叙述']
2024-01-03 15:23:01,832 - INFO : Examples: ['生成与创作', '知识与百科', '知识与百科', None, '故事/叙述+人生经历']


In [8]:

label_df.loc[:,'single_label']=single_label_result_list
label_df.loc[:,'combine_label']=combine_label_result_list
show_df(label_df)


(48301, 8)
  instruction                                             output  \
0  保持健康的三个提示。  以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游...   
1     三原色是什么？  三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在...   

                                               input  \
0  对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。\n参考类别：[计算，逻辑与...   
1  对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。\n参考类别：[计算，逻辑与...   

                                       detail_result               dict_label  \
0  这个指令文本的内容是关于提供保持健康的建议或提示，因此它可以被分类为“生成与创作”类别。这类...          {"类别": "生成与创作"}   
1  这个指令的意图是询问某个特定的知识点，即“三原色”。因此，它应该被归入“知识与百科”类别。在...  {\n    "类别": "知识与百科"\n}   

  label single_label combine_label  
0     生        生成与创作         生成与创作  
1     知        知识与百科         知识与百科  
instruction         0
output              0
input               0
detail_result       0
dict_label       6295
label            6961
single_label     6961
combine_label    6961
dtype: int64


In [11]:

save_df=label_df.loc[label_df["single_label"].notnull(),:].copy()
show_df(save_df)




(41340, 8)
  instruction                                             output  \
0  保持健康的三个提示。  以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游...   
1     三原色是什么？  三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在...   

                                               input  \
0  对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。\n参考类别：[计算，逻辑与...   
1  对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。\n参考类别：[计算，逻辑与...   

                                       detail_result               dict_label  \
0  这个指令文本的内容是关于提供保持健康的建议或提示，因此它可以被分类为“生成与创作”类别。这类...          {"类别": "生成与创作"}   
1  这个指令的意图是询问某个特定的知识点，即“三原色”。因此，它应该被归入“知识与百科”类别。在...  {\n    "类别": "知识与百科"\n}   

  label single_label combine_label  
0     生        生成与创作         生成与创作  
1     知        知识与百科         知识与百科  
instruction      0
output           0
input            0
detail_result    0
dict_label       0
label            5
single_label     0
combine_label    0
dtype: int64


In [13]:

save_mode=True
save_file="output_data/Alpaca_gpt4_zh_instruct_label_data_5w_notnull_v2_240103.xlsx"
if save_mode:
    save_df.to_excel(save_file,index=False)
    logger.info(f"data nums: {save_df.shape}, save file: {save_file}")



2024-01-03 15:31:34,801 - INFO : data nums: (41340, 8), save file: output_data/Alpaca_gpt4_zh_instruct_label_data_5w_notnull_v2_240103.xlsx


In [ ]:
################################
## Stage-3: 针对预测的结果进行格式处理
################################







